In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
import pickle

# **Scraping Data**

In [2]:
url = 'https://www.espncricinfo.com/records/tournament/batting-most-runs-career/icc-cricket-world-cup-2023-24-15338'
all_tables = pd.read_html(url)
current_wc_df = all_tables[0]
print(current_wc_df)

                     Player       Span  Mat  Inns  NO Runs    HS    Ave   BF  \
0             V Kohli (IND)  2023-2023    9     9   3  594  103*  99.00  671   
1            Q de Kock (SA)  2023-2023    9     9   0  591   174  65.66  541   
2           R Ravindra (NZ)  2023-2023    9     9   1  565  123*  70.62  521   
3           RG Sharma (IND)  2023-2023    9     9   0  503   131  55.88  414   
4           DA Warner (AUS)  2023-2023    9     9   0  499   163  55.44  473   
..                      ...        ...  ...   ...  ..  ...   ...    ...  ...   
140      JR Hazlewood (AUS)  2023-2023    9     5   3    4     2   2.00    5   
141  Fazalhaq Farooqi (AFG)  2023-2023    6     3   2    2    2*   2.00    9   
142        LH Ferguson (NZ)  2023-2023    6     2   1    1     1   1.00    6   
143          AT Carey (AUS)  2023-2023    1     1   0    -     0   0.00    2   
144         Usama Mir (PAK)  2023-2023    4     1   0    -     0   0.00    3   

         SR 100 50  0  4s  6s  
0     8

In [3]:
url = 'https://www.espncricinfo.com/records/trophy/batting-most-runs-career/world-cup-12'
all_tables = pd.read_html(url)
past_wc_df = all_tables[0]
print(past_wc_df)

                 Player       Span  Mat  Inns  NO  Runs    HS    Ave    BF  \
0    SR Tendulkar (IND)  1992-2011   45    44   4  2278   152  56.95  2560   
1      RT Ponting (AUS)  1996-2011   46    42   4  1743  140*  45.86  2180   
2         V Kohli (IND)  2011-2023   35    35   7  1624   107  58.00  1859   
3    KC Sangakkara (SL)  2003-2015   37    35   8  1532   124  56.74  1770   
4       DA Warner (AUS)  2015-2023   27    27   2  1491   178  59.64  1484   
..                  ...        ...  ...   ...  ..   ...   ...    ...   ...   
145  KR Rutherford (NZ)  1987-1992   14    12   2   416    75  41.60   587   
146       AH Jones (NZ)  1987-1992   13    13   2   416    78  37.81   673   
147     LRD Mendis (SL)  1975-1987   16    16   2   412    64  29.42   591   
148     PN Kirsten (SA)  1992-1992    8     8   2   410    90  68.33   616   
149  Fakhar Zaman (PAK)  2019-2023   12    12   1   406  126*  36.90   406   

         SR 100  50  0   4s  6s  
0     88.98   6  15  2  241  

In [4]:
df1 = pd.DataFrame(current_wc_df)
df2 = pd.DataFrame(past_wc_df)

In [5]:
frames=[df1,df2]
runs_df = pd.concat(frames)

In [6]:
runs_df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,V Kohli (IND),2023-2023,9,9,3,594,103*,99.00,671,88.52,2,5,1,55,7
1,Q de Kock (SA),2023-2023,9,9,0,591,174,65.66,541,109.24,4,-,-,57,21
2,R Ravindra (NZ),2023-2023,9,9,1,565,123*,70.62,521,108.44,3,2,-,52,17
3,RG Sharma (IND),2023-2023,9,9,0,503,131,55.88,414,121.49,1,3,1,58,24
4,DA Warner (AUS),2023-2023,9,9,0,499,163,55.44,473,105.49,2,2,-,48,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,KR Rutherford (NZ),1987-1992,14,12,2,416,75,41.6,587,70.86,-,4,-,36,7
146,AH Jones (NZ),1987-1992,13,13,2,416,78,37.81,673,61.81,-,4,1,44,0
147,LRD Mendis (SL),1975-1987,16,16,2,412,64,29.42,591,69.71,-,3,1,37,4
148,PN Kirsten (SA),1992-1992,8,8,2,410,90,68.33,616,66.55,-,4,-,28,2


In [7]:
runs_df.isna().sum()

Player    0
Span      0
Mat       0
Inns      0
NO        0
Runs      0
HS        0
Ave       0
BF        0
SR        0
100       0
50        0
0         0
4s        0
6s        0
dtype: int64

In [8]:
runs_df.dtypes

Player     object
Span       object
Mat         int64
Inns        int64
NO          int64
Runs       object
HS         object
Ave        object
BF          int64
SR        float64
100        object
50         object
0          object
4s         object
6s         object
dtype: object

In [9]:
runs_df = runs_df.replace('-', '0')

columns_to_clean = ['Mat','Runs','HS','Ave','100','50','0','4s','6s']

for col in columns_to_clean:
    runs_df[col] = runs_df[col].astype(str).str.replace(r'[*+]', '', regex=True)
    runs_df[col] = pd.to_numeric(runs_df[col], errors='coerce')

print(runs_df.isna().sum())

Player    0
Span      0
Mat       0
Inns      0
NO        0
Runs      0
HS        0
Ave       0
BF        0
SR        0
100       0
50        0
0         0
4s        0
6s        0
dtype: int64


In [10]:
runs_df = runs_df.rename(columns={'100': 'Hundreds', '50': 'Fifties', '4s': 'Fours', '6s': 'Sixes'})

In [11]:
features = ['Inns','Ave', 'SR','Hundreds','Fifties','Fours','Sixes']

X = runs_df[features]
y = runs_df['Runs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Linear Regression**

In [12]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [13]:
player_2023 = pd.DataFrame({
    'Inns': [9, 9, 9],
    'Ave': [99.00 , 65.66, 70.62],
    'SR': [88.52, 109.24, 108.44],
    'Hundreds' : [2, 4, 3],
    'Fifties' : [5, 0, 2],
    'Fours' : [55, 57, 52],
    'Sixes' : [7, 21, 17]
})

predicted_runs_2023 = lr_model.predict(player_2023)
player_2023['Predicted_Runs_2023'] = np.ceil(predicted_runs_2023)
print(player_2023)

   Inns    Ave      SR  Hundreds  Fifties  Fours  Sixes  Predicted_Runs_2023
0     9  99.00   88.52         2        5     55      7                667.0
1     9  65.66  109.24         4        0     57     21                617.0
2     9  70.62  108.44         3        2     52     17                601.0


In [14]:
pickle.dump(lr_model, open('lr_model.pkl', 'wb'))

# **ANN**

In [15]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
model = Sequential()

model.add(Dense(256, input_shape=(X.shape[1],), activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
8/8 [==============================] - 3s 79ms/step - loss: 334720.9062 - val_loss: 355821.2812
Epoch 2/100
8/8 [==============================] - 0s 16ms/step - loss: 332630.9062 - val_loss: 352092.6875
Epoch 3/100
8/8 [==============================] - 0s 19ms/step - loss: 327439.1562 - val_loss: 343080.0625
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 315282.9375 - val_loss: 323172.0000
Epoch 5/100
8/8 [==============================] - 0s 32ms/step - loss: 288998.7812 - val_loss: 284140.2812
Epoch 6/100
8/8 [==============================] - 0s 19ms/step - loss: 243990.9688 - val_loss: 216363.1094
Epoch 7/100
8/8 [==============================] - 0s 17ms/step - loss: 168393.5469 - val_loss: 123393.6797
Epoch 8/100
8/8 [==============================] - 0s 15ms/step - loss: 86159.3906 - val_loss: 43175.4141
Epoch 9/100
8/8 [==============================] - 0s 15ms/step - loss: 35522.1484 - val_loss: 31818.5254
Epoch 10/100
8/8 [==============

In [17]:
player_2023 = pd.DataFrame({
    'Inns': [9, 9, 9],
    'Ave': [99.00 , 65.66, 70.62],
    'SR': [88.52, 109.24, 108.44],
    'Hundreds' : [2, 4, 3],
    'Fifties' : [5, 0, 2],
    'Fours' : [55, 57, 52],
    'Sixes' : [7, 21, 17]
})

player_2023_scaled = sc.transform(player_2023)

predicted_runs_2023 = model.predict(player_2023_scaled)
player_2023['Predicted_runs_2023'] = np.ceil(predicted_runs_2023)
print(player_2023)

1/1 [==============================] - 0s 105ms/step
   Inns    Ave      SR  Hundreds  Fifties  Fours  Sixes  Predicted_runs_2023
0     9  99.00   88.52         2        5     55      7                661.0
1     9  65.66  109.24         4        0     57     21                609.0
2     9  70.62  108.44         3        2     52     17                554.0


In [18]:
with open('scaler_runs.pkl', 'wb') as scaler_file:
    pickle.dump(sc, scaler_file)

In [19]:
model.save('runs_NN_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
